# W2 Form Extraction - Base Model Evaluation

Evaluate Amazon Nova Lite's base performance on W2 tax form extraction before fine-tuning.
This establishes the baseline accuracy we aim to improve through fine-tuning.

In this notebook we will:

- Download the W2 tax form dataset and upload images to S3
- Test the base model on a single sample with DeepDiff comparison
- Run a full evaluation on 100 test samples to measure baseline accuracy

## Environment Setup

In [ ]:
%pip install --upgrade pip --quiet
%pip install boto3 datasets pillow tqdm ipywidgets deepdiff --upgrade --quiet

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import json
import warnings
warnings.filterwarnings("ignore")

from util import *

clients = get_aws_clients()
session     = clients["session"]
s3_client   = clients["s3"]
account_id  = clients["account_id"]
bedrock_runtime = clients["bedrock_runtime"]

bucket_name = f"nova-vision-ft-{account_id}-{REGION}"

print(f"Account ID: {account_id}")
print(f"Bucket name: {bucket_name}")

## Create S3 Bucket and Download Dataset

In [ ]:
create_s3_bucket(s3_client, bucket_name)

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("singhsays/fake-w2-us-tax-form-dataset", split="train")
test_dataset  = load_dataset("singhsays/fake-w2-us-tax-form-dataset", split="test")

print(f"Training examples:   {len(train_dataset)}")
print(f"Test examples:       {len(test_dataset)}")

## Upload Images to S3

Upload all splits now so they are available for both evaluation and later fine-tuning data preparation.

In [ ]:
train_s3_paths = upload_images_to_s3(s3_client, train_dataset, bucket_name, "train")
test_s3_paths  = upload_images_to_s3(s3_client, test_dataset,  bucket_name, "test")

print(f"\nUploaded {len(train_s3_paths)} train, {len(test_s3_paths)} test images")

## Base Model - Single Sample Inference

Test the base Nova Lite model on a single W2 form to see how it performs before fine-tuning.

In [7]:
nova_lite_id = "us.amazon.nova-2-lite-v1:0"

messages = build_inference_messages(test_s3_paths[0]["s3_uri"], account_id)

response = bedrock_runtime.converse(
    modelId=nova_lite_id,
    messages=messages,
    inferenceConfig={"maxTokens": 2048, "temperature": 0.0},
)

response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)

```json
{
    "employee": {
        "name": "Diana Reyes",
        "address": "094 Harris Prairie Susannville ME 60154-1359",
        "socialSecurityNumber": "053-93-7915"
    },
    "employer": {
        "name": "West and Sons Inc",
        "ein": "38-0226974",
        "address": "0324 Morgan Brook Port Shawstand KY 78445-9845"
    },
    "earnings": {
        "wages": 126589.34,
        "socialSecurityWages": 122867.85,
        "medicareWagesAndTips": 114182.15,
        "federalIncomeTaxWithheld": 43873.99,
        "stateIncomeTax": 9392.39,
        "localWagesTips": 101209.95,
        "localIncomeTax": 14120.87
    },
    "benefits": {
        "dependentCareBenefits": 114182.15,
        "nonqualifiedPlans": 158
    },
    "multiStateEmployment": {
        "WI": {
            "localWagesTips": 125139.92,
            "localIncomeTax": 23035.86,
            "localityName": "William Canyon"
        },
        "HI": {
            "localWagesTips": 101209.95,
            "localIncomeTax":

In [8]:
from deepdiff import DeepDiff

prediction = parse_json_from_markdown(response_text)
gt = transform_schema(json.loads(test_s3_paths[0]["gt"])["gt_parse"])

print("=== DeepDiff: Ground Truth vs. Prediction ===")
diff = DeepDiff(gt, prediction, ignore_order=True)
diff

=== DeepDiff: Ground Truth vs. Prediction ===


{'type_changes': {"root['benefits']['dependentCareBenefits']": {'old_type': int,
   'new_type': float,
   'old_value': 219,
   'new_value': 114182.15}},
 'values_changed': {"root['employee']['address']": {'new_value': '094 Harris Prairie Susannville ME 60154-1359',
   'old_value': '094 Harris Prairie, Susanville ME 60154-1359'},
  "root['employer']['address']": {'new_value': '0324 Morgan Brook Port Shawstand KY 78445-9845',
   'old_value': '0324 Morgan Brook, Port Shawnstad KY 78445-9845'},
  "root['earnings']['stateIncomeTax']": {'new_value': 9392.39,
   'old_value': 11757.36},
  "root['multiStateEmployment']['HI']['localityName']": {'new_value': 'Spokane Junction',
   'old_value': 'Cynthia Junctions'}}}

## Full Base Model Evaluation (100 test samples)

Run the base model on all 100 test samples and compute accuracy metrics by field category.

In [9]:
test_data = build_test_data_in_memory(test_s3_paths, account_id)

print("Running base model evaluation on 100 test samples...")
base_eval_results = evaluate_model_on_test_data(bedrock_runtime, test_data, 100, nova_lite_id)

so = base_eval_results["structured_output"]
print(f"\nStructured Output (Valid JSON):")
print(f"  Parse success rate: {so['parse_success_rate']:.2%} ({so['parse_successes']}/{so['total']})")
print(f"  Parse failures:     {so['parse_failures']}")

print(f"\nOverall Field Extraction Accuracy: {base_eval_results['overall_accuracy']:.2%}")
print("\nAccuracy by Field Category:")
for category, accuracy in base_eval_results["category_accuracies"].items():
    print(f"  - {category}: {accuracy:.2%}")

Running base model evaluation on 100 test samples...


Evaluating model: 100%|██████████| 100/100 [04:41<00:00,  2.82s/it]


Structured Output (Valid JSON):
  Parse success rate: 100.00% (100/100)
  Parse failures:     0

Overall Field Extraction Accuracy: 51.90%

Accuracy by Field Category:
  - Employee Information: 60.00%
  - Employer Information: 48.00%
  - Earnings: 42.57%
  - Benefits: 50.00%
  - Multi-State Employment: 62.91%
  - Other: 0.00%


## Save Variables

Persist variables needed by subsequent notebooks.

In [ ]:
%store bucket_name
%store account_id
%store train_s3_paths
%store val_s3_paths
%store test_s3_paths
%store base_eval_results

print("Variables saved for subsequent notebooks")